In [1]:
import sys
sys.path.append("..")

import pyadlml.dataset.kasteren as kasteren
from pyadlml.dataset._dataset import ACTIVITY, END_TIME, START_TIME
import pandas as pd
import pyadlml.dataset.plotly.activities as plt_act
import pyadlml.dataset.plotly.devices as plt_dev
import matplotlib.pyplot as plt

In [2]:
import plotly
plotly.offline.init_notebook_mode(connected=True)

In [3]:
sensorData = "/home/chris/code/adlml/datasets/kasteren/kasterenSenseData.txt"
activityData = "/home/chris/code/adlml/datasets/kasteren/kasterenActData.txt"

data = kasteren.load(sensorData, activityData)

In [4]:
data.df_activities.head()

,start_time,end_time,activity
0,2008-02-25 19:40:26.000,2008-02-25 20:22:58,prepare Dinner
1,2008-02-25 20:23:12.000,2008-02-25 20:23:35,get drink
2,2008-02-25 21:51:29.000,2008-02-25 21:52:36,use toilet
3,2008-02-25 23:21:15.000,2008-02-25 23:28:30,go to bed
4,2008-02-25 23:28:30.001,2008-02-25 23:29:14,use toilet


In [5]:
data.df_dev_rep2.head()

,start_time,end_time,val,device
0,2008-02-25 00:20:14,2008-02-25 00:22:57,True,Hall-Bedroom door
1,2008-02-25 09:33:41,2008-02-25 09:33:42,True,Hall-Bedroom door
2,2008-02-25 09:33:47,2008-02-25 17:21:12,True,Hall-Bedroom door
3,2008-02-25 09:36:43,2008-02-25 09:37:04,True,Hall-Toilet door
4,2008-02-25 09:37:20,2008-02-25 09:37:23,True,Hall-Bathroom door


In [6]:
data.df_devices.head()

,Cups cupboard,Dishwasher,Freezer,Fridge,Frontdoor,Groceries Cupboard,Hall-Bathroom door,Hall-Bedroom door,Hall-Toilet door,Microwave,Pans Cupboard,Plates cupboard,ToiletFlush,Washingmachine
time,,,,,,,,,,,,,,
2008-02-25 00:20:14,0,0,0,0,0,0,0,1,0,0,0,0,0,0
2008-02-25 00:22:57,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2008-02-25 09:33:41,0,0,0,0,0,0,0,1,0,0,0,0,0,0
2008-02-25 09:33:42,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2008-02-25 09:33:47,0,0,0,0,0,0,0,1,0,0,0,0,0,0


In [7]:
plt_act.gantt(data.df_activities).show()

In [8]:
plt_act.hist_counts(data.df_activities).show()

In [9]:
plt_act.boxplot_duration(data.df_activities, y_scale='log').show()

In [10]:
plt_act.hist_cum_duration(data.df_activities).show()#, y_scale='log').show()

In [11]:
plt_act.ridge_line(data.df_activities, t_range='day', n=1000).show()

#### Transition Heatmap
Can be read as firt used the toilet and then got into bed 46 times

In [12]:
plt_act.heatmap_transitions(data.df_activities).show()

In [13]:
plt_dev.hist_on_off(data.df_dev_rep2.copy()).show()

In [14]:
t_res=['1m', '10m', '30m', '1h', '2h', '6h']
plt_dev.heatmap_trigger_one_day(data.df_dev_rep2, t_res).show()

In [15]:
#t_windows = ['5s', '10s', '30s']
t_windows = '5s'
plt_dev.heatmap_time_diff(data.df_dev_rep2.copy(), t_windows).show()

nr:  1


In [30]:
import pyadlml.dataset.plot.devices as mplot_dev
#mplot_dev.hist_trigger_time_diff(data.df_dev_rep2);

In [31]:
plt_dev.hist_trigger_time_diff(data.df_dev_rep2).show()

In [29]:
#hist_counts(data.df_dev_rep2, scale_y='log')
#hist_counts(data.df_dev_rep2, scale_y='norm')
plt_dev.hist_counts(data.df_dev_rep2)

# Representations

In [32]:
# creates raw data from the two dataframes
data.create_raw()
data.df_raw.head()

,Cups cupboard,Dishwasher,Freezer,Fridge,Frontdoor,Groceries Cupboard,Hall-Bathroom door,Hall-Bedroom door,Hall-Toilet door,Microwave,Pans Cupboard,Plates cupboard,ToiletFlush,Washingmachine,activity
time,,,,,,,,,,,,,,,
2008-02-25 00:20:14,0,0,0,0,0,0,0,1,0,0,0,0,0,0,NaT
2008-02-25 00:22:57,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaT
2008-02-25 09:33:41,0,0,0,0,0,0,0,1,0,0,0,0,0,0,NaT
2008-02-25 09:33:42,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaT
2008-02-25 09:33:47,0,0,0,0,0,0,0,1,0,0,0,0,0,0,NaT


In [33]:
df = data.df_raw
num_rows = len(df[ACTIVITY])
nan_amount = num_rows - df[ACTIVITY].count()
fraction = nan_amount/num_rows

print('total nan values:  ', nan_amount)
print('fraction of all: ', fraction)

# print length of zusammenhaengende sequenze max min 
for row in df.iterrows():
    pass

total nan values:   606
fraction of all:  0.22971948445792267


# creates sequential data from the raw dataframe
data.createSequential()
data.df_seq

# convertes sequential data into the last fired representation
data.conv2lastFired()
data.df_lf

# convertes sequential data into the last fired representation
data.conv2Changed()
data.df_ch